# Relaxing Model

Este es un modelo de ahorros para retiro que modela el salario con crecimiento constante para la tasa de costo de vida y los incrementos regulares para promociones. El modelo está partido en las siguientes secciones:

- [**Estructura**](#Estructura): Ejecuta cualquier import y otras configuraciones.
- [**Inputs**](#Inputs): Define los inputs del modelo.
- [**Salarios**](#Salarios): Determina el salario cada año, considerando los aumentos en costo de vida y promociones.
- [**Ahorros**](#Ahorros): Determina la riqueza en cada año, considerando una tasa constante de ahorros y una tasa de de   inversión.
- [**Retiro**](#Retiro): Determina los años para retirarse desde los ahorros en el tiempo, el principal output del modelo.

## Estructura

La configuración para los calculos posteriores se presenta acá. Importamos los módulos necesarios:

In [1]:
# Importamos dataclasses para habilitar el objeto dataclass
from dataclasses import dataclass
import numpy as np
import pandas as pd
%matplotlib inline

## Inputs

Todas las entradas del modelo se definen aquí. Se construye una clase tipo dataclass para administrar los datos y se crea una instancia de la clase que contiene las entradas predeterminadas.

In [2]:
# Escribimos el decorator de sintaxis
@dataclass
class ModelInputs:
    starting_salary: float= 60000
    promos_every_n_years: int= 5
    cost_of_living_raise: float= 0.02
    promos_raise: float= 0.15
    savings_rate: float= 0.25
    interest_rate: float= 0.05
    annual_spend_in_retirement: float= 40000 # Parte del ejercicio
    years_in_retirement: int= 20 # Parte del ejercicio



# Almacenamos la clase en una variable model_data
model_data = ModelInputs()
model_data  

ModelInputs(starting_salary=60000, promos_every_n_years=5, cost_of_living_raise=0.02, promos_raise=0.15, savings_rate=0.25, interest_rate=0.05, annual_spend_in_retirement=40000, years_in_retirement=20)

In [3]:
data= model_data

## Salarios

Aquí se calcula el salario de cada año. Suponemos que el salario crece a una tasa constante cada año por los aumentos del costo de vida, y luego también cada número de años, el salario aumenta en un porcentaje adicional debido a una promoción o cambio de trabajo. Con base en este supuesto, el salario evolucionaría con el tiempo con la siguiente ecuación:

$$s_t = s_0 (1 + r_{cl})^n (1 + r_p)^p$$

Donde:

- $s_t$: Salario en el año $t$
- $s_0$: Salario inicial (en el año 0)
- $r_{cl}$: Costo de vida anual
- $r_p$: Tasa de promociones
- $p$: Número de promociones

En Python, se ve así:

In [4]:
# Creamos una función salario
def salary_at_year(data: ModelInputs, year):
    """
    Obtiene el salario en un año determinado desde el inicio del modelo en función de los aumentos del costo de vida 
    y las promociones regulares.
    
    """
    # Cada cierto número de años hay una promoción, por lo tanto, dividimos los años y quitamos los decimales para conseguir
    # el número de promociones
    num_promos = int(year / data.promos_every_n_years) 
    
    # Llevamos a cabo la fórmula anteriormente mencionada
    salario_t = data.starting_salary * (1 + data.cost_of_living_raise) ** year * (1 + data.promos_raise) ** num_promos
    return salario_t

Esa función obtendrá el salario en un año determinado, por lo que para obtener todos los salarios, simplemente la ejecutamos cada año. Pero no sabremos cuántos años funcionará como deberíamos hacerlo hasta que el individuo pueda jubilarse. Por lo tanto, solo mostramos los primeros salarios por ahora y luego usaremos esta función en la sección de [Ahorros](Ahorros) del modelo.

In [5]:
# Determinamos los salarios en los primeros 6 años
for i in range(6):
    year = i + 1
    salario = salary_at_year(model_data, year)
    print(f'El salario en el año {year} es ${salario:,.0f}.')

El salario en el año 1 es $61,200.
El salario en el año 2 es $62,424.
El salario en el año 3 es $63,672.
El salario en el año 4 es $64,946.
El salario en el año 5 es $76,182.
El salario en el año 6 es $77,705.


Como era de esperar, con los insumos predeterminados, el salario aumenta a un 2% anual. Luego, en el año 5, hay una promoción por lo que hay un aumento mayor en el salario.

## Ahorros

La parte de ahorro del modelo se ocupa de aplicar la tasa de ahorro al salario ganado para calcular el efectivo ahorrado, acumular el efectivo ahorrado a lo largo del tiempo y aplicar la tasa de inversión a la riqueza acumulada.

Para calcular el efectivo ahorrado, es simplemente:

$$c_t = s_t * r_s$$

Donde:
- $c_t$: Efectivo ahorrado durante el año $t$
- $r_s$: Tasa de ahorros

In [6]:
# Definimos la función de Efectivo ahorrado
def cash_saved_during_year(data: ModelInputs, year):
    """
    Cálculo del efectivo ahorrado dentro de un año dado, calculando primero el salario en ese año y luego aplicando el
    tasa de ahorro.
    """
    salario = salary_at_year(data, year) # Determinamos el salario en el año n
    ahorro = salario * data.savings_rate
    return ahorro

Para obtener la riqueza de cada año, solo se aplica el rendimiento de la inversión al ahorro del año pasado y luego se agrega el efectivo ahorrado de este año:

$$w_t = w_{t-1} (1 + r_i) + c_t$$

Donde:
- $w_t$: Ahorro en el año $t$
- $r_i$: Tasa de inversión

In [7]:
# Creamos la función de ahorros en el año n
def wealth_at_year(data: ModelInputs, year, prior_savings):
    """
    Calcula la riqueza acumulada para un año dado, basándose en la riqueza anterior, la tasa de inversión,
     y efectivo ahorrado durante el año.
    """
    ahorro = cash_saved_during_year(data, year)
    savings =  prior_savings * (1 + data.interest_rate) + ahorro
    return savings

Nuevamente, al igual que en la sección [Salarios](Salarios), ahora podemos obtener el resultado de cada año, pero no sabemos en última instancia cuántos años tendremos para ejecutarlo. Eso se determinará en la sección de [Retiro](Retiro). Entonces, por ahora, solo muestre los primeros años de acumulación de riqueza:

In [8]:
# Determinamos los ahorros
prior_savings= 0 # Empezamos de cero
for i in range(6):
    year = i + 1
    ahorros= wealth_at_year(model_data, year, prior_savings)
    print(f'Los ahorros en el año {year} fueron de ${ahorros:,.0f}.')
    
    # Establecer la riqueza anterior del año próximo a la riqueza de este año
    prior_savings = ahorros

Los ahorros en el año 1 fueron de $15,300.
Los ahorros en el año 2 fueron de $31,671.
Los ahorros en el año 3 fueron de $49,173.
Los ahorros en el año 4 fueron de $67,868.
Los ahorros en el año 5 fueron de $90,307.
Los ahorros en el año 6 fueron de $114,248.


Con insumos predeterminados, la riqueza aumenta aproximadamente en un 25% del salario cada año, más un poco más para la inversión. Luego, en el año 6, vemos un aumento sustancialmente mayor porque el salario es sustancialmente mayor debido a la promoción. Entonces todo parece correcto.

## Retiro

Esta sección del modelo reúne todo para producir el resultado final de los años hasta la jubilación. Utiliza la lógica para obtener los ahorros en cada año, que a su vez usa la lógica para obtener el salario en cada año. El ahorro de cada año se rastrea a lo largo del tiempo hasta que alcanza el efectivo deseado. Una vez que la riqueza alcanza el efectivo deseado, el individuo puede jubilarse, de modo que ese año se devuelve como años hasta la jubilación.


## Calcular el efectivo deseado


In [9]:
# Inspeccionamos la fórmula de valor presente
help(np.pv)  # Opcional

Help on function pv in module numpy:

pv(rate, nper, pmt, fv=0, when='end')
    Compute the present value.
    
    .. deprecated:: 1.18
    
       `pv` is deprecated; for details, see NEP 32 [1]_.
       Use the corresponding function in the numpy-financial library,
       https://pypi.org/project/numpy-financial.
    
    Given:
     * a future value, `fv`
     * an interest `rate` compounded once per period, of which
       there are
     * `nper` total
     * a (fixed) payment, `pmt`, paid either
     * at the beginning (`when` = {'begin', 1}) or the end
       (`when` = {'end', 0}) of each period
    
    Return:
       the value now
    
    Parameters
    ----------
    rate : array_like
        Rate of interest (per period)
    nper : array_like
        Number of compounding periods
    pmt : array_like
        Payment
    fv : array_like, optional
        Future value
    when : {{'begin', 1}, {'end', 0}}, {string, int}, optional
        When payments are due ('begin' (1) or 

In [10]:
# Determinamos el valor presente de acuerdo con las condiciones del ejercicio
def get_desired_cash(data):
    return np.pv(data.interest_rate, data.years_in_retirement, -data.annual_spend_in_retirement)

get_desired_cash(model_data)

print(f'Martha necesita ${get_desired_cash(model_data):,.0f} para el retiro.')

Martha necesita $498,488 para el retiro.


<ipython-input-10-c02f704f9e98>:3: DeprecationWarning: numpy.pv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pv instead (https://pypi.org/project/numpy-financial/).
  return np.pv(data.interest_rate, data.years_in_retirement, -data.annual_spend_in_retirement)


## Calcular años para el retiro

In [11]:
# Creamos la función años para retiro
def years_to_retirement(data: ModelInputs):
    
    # Iniciamos con cero 
    prior_savings = 0  
    ahorros = 0
    
    year = 0  # Se volverá 1 en el primer ciclo del loop
    
    desired_cash = get_desired_cash(data)
    
    print('Ahorros en el tiempo:') 
    while ahorros < desired_cash:
        year = year + 1
        ahorros= wealth_at_year(model_data, year, prior_savings)
        print(f'Los ahorros en el año {year} fueron ${ahorros:,.0f}.')

        #  Establece el ahorro previo como la riqueza de este año
        prior_savings = ahorros
        
    # Ahora hemos salido del ciclo while, por lo que la riqueza debe ser> = deseado_en efectivo. 
    # Lo que sea que estableció el año t-1 son los años necesarios para retirarse. 
    print(f'\nRetiro:\nTomará {year} años retirarse.') 
    return year

Con los inputs por default:

In [12]:
años = years_to_retirement(ModelInputs(years_in_retirement= 25))

Ahorros en el tiempo:
Los ahorros en el año 1 fueron $15,300.
Los ahorros en el año 2 fueron $31,671.
Los ahorros en el año 3 fueron $49,173.
Los ahorros en el año 4 fueron $67,868.
Los ahorros en el año 5 fueron $90,307.
Los ahorros en el año 6 fueron $114,248.
Los ahorros en el año 7 fueron $139,775.
Los ahorros en el año 8 fueron $166,975.
Los ahorros en el año 9 fueron $195,939.
Los ahorros en el año 10 fueron $229,918.
Los ahorros en el año 11 fueron $266,080.
Los ahorros en el año 12 fueron $304,542.
Los ahorros en el año 13 fueron $345,431.
Los ahorros en el año 14 fueron $388,878.
Los ahorros en el año 15 fueron $439,025.
Los ahorros en el año 16 fueron $492,294.
Los ahorros en el año 17 fueron $548,853.
Los ahorros en el año 18 fueron $608,878.

Retiro:
Tomará 18 años retirarse.


<ipython-input-10-c02f704f9e98>:3: DeprecationWarning: numpy.pv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pv instead (https://pypi.org/project/numpy-financial/).
  return np.pv(data.interest_rate, data.years_in_retirement, -data.annual_spend_in_retirement)


In [13]:
años = years_to_retirement(model_data)

Ahorros en el tiempo:
Los ahorros en el año 1 fueron $15,300.
Los ahorros en el año 2 fueron $31,671.
Los ahorros en el año 3 fueron $49,173.
Los ahorros en el año 4 fueron $67,868.
Los ahorros en el año 5 fueron $90,307.
Los ahorros en el año 6 fueron $114,248.
Los ahorros en el año 7 fueron $139,775.
Los ahorros en el año 8 fueron $166,975.
Los ahorros en el año 9 fueron $195,939.
Los ahorros en el año 10 fueron $229,918.
Los ahorros en el año 11 fueron $266,080.
Los ahorros en el año 12 fueron $304,542.
Los ahorros en el año 13 fueron $345,431.
Los ahorros en el año 14 fueron $388,878.
Los ahorros en el año 15 fueron $439,025.
Los ahorros en el año 16 fueron $492,294.
Los ahorros en el año 17 fueron $548,853.

Retiro:
Tomará 17 años retirarse.


<ipython-input-10-c02f704f9e98>:3: DeprecationWarning: numpy.pv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pv instead (https://pypi.org/project/numpy-financial/).
  return np.pv(data.interest_rate, data.years_in_retirement, -data.annual_spend_in_retirement)
